<a href="https://colab.research.google.com/github/Griznt/colab_test_tasks/blob/main/Netology_test_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Задача 1

Составить sql запрос по задаче:
Сделать выгрузку по терминалам, в которых в течение короткого периода времени (временного окна) провели оплаты несколько клиентов

**Входные данные:**

* Период выгрузки с сентября 2019 по конец января 2020
* Временное окно операций - 2 часа
* Кол-во уникальных клиентов, которые совершили операции в «окне» - 3 и более клиентов

**Какие поля содержит рабочая таблица (operations):**

`id` - индекс (номер операции), тип int

`amount` - сумма операции, тип float

`user_id` - номер клиента, тип int

`term_id` - номер банкомата, тип int

`created_at` - дата операции, тип time stamp

На выходе должна получиться таблица со списком терминалов и операций, удовлетворяющих условию.



Задачу плохо понял. Если надо сделать на чистом SQL ни разу такое не делал, максимум - генерировал даты. Могу сделать только на Python.

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import requests
import os
import json
import urllib
from datetime import datetime, timedelta, date

In [ ]:
con = sqlite3.connect('db')

In [ ]:
def select(sql):
  return pd.read_sql(sql, con)

попробуем решить задачу на python.


Создадим диапазон дат. Наполним каждую дату временем так, чтобы в каждом окне 2ч было не менее 3х временных промежутков.

In [ ]:
24/2 


12.0

всего 12 промежутков

In [ ]:
120/3


40.0

в одном окне в 2ч должно быть не менее 3х транзакций.

Значит каждые 2 ч с периодичностью <= 40 минут.

Сделаем промежуток 24 минуты.

In [ ]:
df = pd.DataFrame({'created_at': pd.date_range('2019-09-01T00:00:01Z', '2020-01-31T23:59:59Z', freq='24T') })
df.describe()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  


,created_at
count,9180
unique,9180
top,2020-01-06 15:12:01+00:00
freq,1
first,2019-09-01 00:00:01+00:00
last,2020-01-31 23:36:01+00:00


Теперь заполним остальное. Важно, что в одном окне должно быть не менее **3х уникальных клиентов**!

Для этого разобьем наши интервалы на диапазоны и в каждом диапазоне создадим уникальных клиентов

In [ ]:
120/15

8.0

С частотой 15 минут, у нас около 8 промежутков внутри 2х часового окна.

Теперь заполним уникальными user_id для каждых 8 значений записей в рамках одной сессии

Создадим новый столбец user_id
 

In [ ]:
df.insert(len(df.columns),'user_id', '')

In [ ]:
df.head()

,created_at,user_id
0,2019-09-01 00:00:01+00:00,
1,2019-09-01 00:24:01+00:00,
2,2019-09-01 00:48:01+00:00,
3,2019-09-01 01:12:01+00:00,
4,2019-09-01 01:36:01+00:00,


In [ ]:
import random
def get_unique_ids():
  return random.sample(range(1000, 2000), 1000)

В цикле пройдемся по элементам датасета. Для каждых 8 значений будем брать уникальные id из диапазона 1000...2000. 

Пусть всего возможно 1000 уникальных user_id.

In [ ]:
unique_ids = get_unique_ids()
part = 8
for ind in df.index:
  if ind == 0:
    df['user_id'][ind] = unique_ids[part]
  elif ind < part:
    df['user_id'][ind] = unique_ids[part - ind]
  elif ind == part:
    df['user_id'][ind] = unique_ids[part - ind]
    part = part + 8
    unique_ids = get_unique_ids()

Проверим, что получилось

In [ ]:
df.describe()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


,created_at,user_id
count,9180,9180.0
unique,9180,1000.0
top,2020-01-06 15:12:01+00:00,1244.0
freq,1,23.0
first,2019-09-01 00:00:01+00:00,NaN
last,2020-01-31 23:36:01+00:00,NaN


9180 записей, где 1000 уникальных id

Теперь оставлось создать другие данные

In [ ]:
df.insert(len(df.columns), 'term_id', range(880, 880 + len(df)))
df.head()

,created_at,user_id,term_id
0,2019-09-01 00:00:01+00:00,1257,880
1,2019-09-01 00:24:01+00:00,1236,881
2,2019-09-01 00:48:01+00:00,1460,882
3,2019-09-01 01:12:01+00:00,1629,883
4,2019-09-01 01:36:01+00:00,1799,884


In [ ]:
df.insert(len(df.columns), 'amount', random.sample(range(20, 20000), len(df)))
df.head()

,created_at,user_id,term_id,amount
0,2019-09-01 00:00:01+00:00,1257,880,3839
1,2019-09-01 00:24:01+00:00,1236,881,11266
2,2019-09-01 00:48:01+00:00,1460,882,11929
3,2019-09-01 01:12:01+00:00,1629,883,18227
4,2019-09-01 01:36:01+00:00,1799,884,16129


In [ ]:
df.insert(len(df.columns), 'id', range(0, len(df)))
df.head()

,created_at,user_id,term_id,amount,id
0,2019-09-01 00:00:01+00:00,1257,880,3839,0
1,2019-09-01 00:24:01+00:00,1236,881,11266,1
2,2019-09-01 00:48:01+00:00,1460,882,11929,2
3,2019-09-01 01:12:01+00:00,1629,883,18227,3
4,2019-09-01 01:36:01+00:00,1799,884,16129,4


готово! Теперь запишем это в БД

In [ ]:
df.to_sql('operations',con,if_exists='replace',index=False)

Проверим, что получилось:

In [ ]:
sql = '''select o.* from operations o limit 10'''

In [ ]:
select(sql)

,created_at,user_id,term_id,amount,id
0,2019-09-01 00:00:01+00:00,1257,880,3839,0
1,2019-09-01 00:24:01+00:00,1236,881,11266,1
2,2019-09-01 00:48:01+00:00,1460,882,11929,2
3,2019-09-01 01:12:01+00:00,1629,883,18227,3
4,2019-09-01 01:36:01+00:00,1799,884,16129,4
5,2019-09-01 02:00:01+00:00,1650,885,2261,5
6,2019-09-01 02:24:01+00:00,1703,886,11659,6
7,2019-09-01 02:48:01+00:00,1555,887,5305,7
8,2019-09-01 03:12:01+00:00,1593,888,3016,8
9,2019-09-01 03:36:01+00:00,1473,889,4108,9


Готово!

#Задача 3

Используя язык программирования Python автоматизировать выгрузку данных из API системы трекинга мобильных установок.


Входные данные:
токен = 65196fe7aafb
временной диапазон - за последние 60 дней, но со вчерашнего дня
поля для выгрузки - attributed_touch_time, install_time, media_source, campaign, customer_user_id, appsflyer_id
ссылка для выгрузки данных - 'https://hq.appsflyer.com/export/ru.foxford.foxfordtextbook/installs_report/v5?api_token=&from=&to=&fields='

Сырые данные нужно обогатить полями:
Неделя установки
Месяц установки
Год установки
CTIT - в секундах, считается как разница между временем установки и кликом на рекламное размещение
CTIT type - группировка по критериям: меньше 10 сек, от 10 до 30 сек, и более 30 сек.

Первично собранный набор данных нужно записать в БД и при повторной реализации скрипта в таблицу должны записываться только новые данные.
Токен не рабочий, поэтому при выполнении запроса будет выдаваться ошибка



Напишем функцию, которая будет осуществлять запрос.

Т.к. единственный параметр который настраивается это api_key - вынесем его в параметры функции.

In [ ]:
def previous_60_days_export(api_token):

  api_endpoint = 'https://hq.appsflyer.com/export/'
  app_id = 'ru.foxford.foxfordtextbook'
  report_name = 'installs_report'

  to_dt = datetime.now() - timedelta(1)
  from_dt = datetime.now() - timedelta(61)

  fileds = ['attributed_touch_time',
            'install_time',
            'media_source',
            'campaign',
            'customer_user_id',
            'appsflyer_id']

  query_params = {
    'api_token': api_token,
    'from': str(from_dt.strftime('%Y-%m-%d')),
    'to': str(to_dt.strftime('%Y-%m-%d')),
    'fields': ','.join(fileds)
    }

  query_string = urllib.parse.urlencode(query_params)

  request_url = api_endpoint + app_id + '/' + report_name + '/v5?' + query_string

  print(request_url)

  response = 0
  try:
    response = requests.get(request_url)
  except requests.exceptions.RequestException as e:
    print(e.message)

  if response:
    return response.json()
  else:
    return  pd.DataFrame(columns=[])

сейчас объявим token и сделаем вызов функции

In [ ]:
api_token = '65196fe7aafb'

df = pd.DataFrame.from_dict(previous_60_days_export(api_token), orient='index')

https://hq.appsflyer.com/export/ru.foxford.foxfordtextbook/installs_report/v5?api_token=65196fe7aafb&from=2021-03-23&to=2021-05-22&fields=attributed_touch_time%2Cinstall_time%2Cmedia_source%2Ccampaign%2Ccustomer_user_id%2Cappsflyer_id


In [ ]:
df.head()

""


Добавим недостающие поля:

In [ ]:
df['install_week'] = pd.DatetimeIndex(df['install_time']).week
df['install_month'] = pd.DatetimeIndex(df['install_time']).month
df['install_year'] = pd.DatetimeIndex(df['install_time']).year


в соответствии с [документацией](https://support.appsflyer.com/hc/ru/articles/208387843-%D0%A1%D0%BB%D0%BE%D0%B2%D0%B0%D1%80%D1%8C-%D0%BF%D0%BE%D0%BB%D0%B5%D0%B9-%D0%B2-%D0%BE%D1%82%D1%87%D0%B5%D1%82%D0%B0%D1%85-%D0%BF%D0%BE-%D1%81%D1%8B%D1%80%D1%8B%D0%BC-%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%BC-V5-) AppsFlyer - считаю событие attributed_touch_time за время клика на рекламное размещение

Я заметил, что разбивка CTIT_type требуется по интервалам 0 до 10, от 10 до 30 (вкл), и более 30, но с ходу знаю как будет лучше. Вряд ли мой вариант лучше стандартного `cut`

In [ ]:
df['CTIT'] = pd.Timedelta(df['install_time'] - df['attributed_touch_time']).seconds
df['CTIT_type'] = [[]] * len(df)
for ind in df.index:
  if df['CTIT'][ind] < 10: 
    df['CTIT_type'][ind] = '[0:10)' 
  elif df['CTIT'][ind] > 30:
    df['CTIT_type'][ind] = '(30:inf)'
  else:
    df['CTIT_type'][ind] = '[10:30]'

# df['CTIT_type'] = pd.cut(df['CTIT'], [0,10,30, np.inf],include_lowest=True,right=False])

Достанем уже имеющиеся данные, отфильтруем то, что пришло и запишем новую порцию данных в бд.

In [ ]:
sql = '''select r.* from report r'''

Сделаем обработку ошибки, когда таблица еще не существует. Если ее нет, то просто запишем все в бд из датасета.

In [ ]:
try:
  select(sql)
except Exception as e:
  print(e, 'Creating table from dataset')
  df.to_sql('report', con,if_exists='append', index=False)

In [ ]:
df.to_sql('report', con,if_exists='append', index=False)


In [ ]:
sql = '''select r.* from report r'''

In [ ]:
stored_data = select(sql)
stored_data.head()

Если данные в БД есть, то нужно сравнить данные и удалить из датасета те, которые уже в БД имеются. То, что останется - залить в бд.

Я рассчитываю, что связка `appsflyer_id, customer_user_id, attributed_touch_time, install_time` является уникальной.
Сложно делать что-то, не видя данных. Из доки AppsFlyer не понятно.

In [ ]:
df['equal'] = np.where(
  df['appsflyer_id'] == stored_data['appsflyer_id'] and
  df['customer_user_id'] == stored_data['customer_user_id'] and
  df['attributed_touch_time'] == stored_data['attributed_touch_time'] and
  df['install_time'] == stored_data['install_time']
  , 1, 0)

Теперь уберем все записи, имеющие метку `equal` и запишем новое в бд

In [ ]:
df[df['equal'] < 1]
if len(df) > 0
  df.to_sql('report', con,if_exists='append', index=False)

Конец